In [34]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(SCRIPT_DIR)

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tueplots import bundles, figsizes

from bocpd import bocpd
from hazard import ConstantHazard
from models import DSMGaussian
from models import Gaussian
from omega_estimator import OmegaEstimatorGaussian
from utils.find_cp import find_cp

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

In [123]:
def accuracy_delay(cps_pred, cps_true, window=10):
    TP = 0
    FP = 0
    FN = 0
    delays = []
    for cp in cps_pred[1:]:
        if np.min(np.abs(np.array(cps_true)-cp))<window:
            TP+=1
            delays.append(np.min(np.abs(np.array(cps_true)-cp)))
        else:
            FP+=1
    for cp in np.array(cps_true)[:-1]:
        if np.min(np.abs(np.array(cps_pred)-cp))>window:
            FN+=1
    return [TP/(TP+FP), TP/(TP+FN)], delays

In [124]:
np.random.seed(3000)

T = 500
data = np.zeros((T,1))
cps_true = [100, 200, 300, 350, 400, 450, 600]
mus = [10, 5, 0 , 10, 5 , 12, 0]
mu = mus[0]
i = 0 
for t in range(0, T):
    if t == cps_true[i]:
        mu = mus[i+1]
        i = i+1
    data[t,0] = np.random.normal(mu, 1)

In [125]:
mean0 = np.mean(data)
var0 = np.var(data)


In [126]:
model  = Gaussian(mu0=mean0, kappa0=1, alpha0=1, omega0=1)
hazard = ConstantHazard(300)
R_ST_NO = bocpd(data, hazard, model)
cps_ST_NO = find_cp(R_ST_NO)

In [127]:
mean_mu0 = mean0
var_mu0 = 1

mean_Sigma0 = 1
var_Sigma0 = 1

mu0 = np.array([[mean_mu0/var_mu0], [1/var_mu0]])

Sigma0 = np.eye(2)
Sigma0[0,0] = mean_Sigma0/var_Sigma0
Sigma0[1,1] = 1/var_Sigma0

def m(x):
    return np.array([(1+x**2)**(-1/2)])

def grad_m(x):
    return np.array([[-x/((1+x**2)**(3/2))]])

model  = DSMGaussian(data=data, m=m, grad_m=grad_m, omega=1, mu0=mu0, Sigma0=Sigma0)
hazard = ConstantHazard(300)
R_DSM_NO = bocpd(data, hazard, model)
cps_DSM_NO = find_cp(R_DSM_NO)

In [128]:
PPV_DSM_NO, delays_DSM_NO = accuracy_delay(cps_DSM_NO,cps_true)
PPV_ST_NO, delays_ST_NO = accuracy_delay(cps_ST_NO,cps_true)
print(PPV_DSM_NO, np.mean(delays_DSM_NO), np.var(delays_DSM_NO))
print(PPV_ST_NO, np.mean(delays_ST_NO), np.var(delays_ST_NO))

[1.0, 1.0] 0.16666666666666666 0.13888888888888892
[1.0, 1.0] 0.0 0.0


In [129]:
np.random.seed(54321)
i_obs = np.random.choice(
    np.arange(0, T, 1), int(0.02 * T), replace=False)

data_contaminated = data.copy()
j = np.random.choice([1,-1],size = len(i_obs))
data_contaminated[i_obs,0] = data_contaminated[i_obs,0] + j*10

In [130]:
mean0 = np.mean(data)

In [142]:
cps_DSM = []
cps_ST = []
for i in range(10):
    i_obs = np.random.choice(
        np.arange(0, T, 1), int(0.02 * T), replace=False)

    data_contaminated = data.copy()
    j = np.random.choice([1,-1],size = len(i_obs))
    data_contaminated[i_obs,0] = data_contaminated[i_obs,0] + j*10

    mean_mu0 = mean0
    var_mu0 = 1

    mean_Sigma0 = 1
    var_Sigma0 = 1

    mu0 = np.array([[mean_mu0/var_mu0], [1/var_mu0]])

    Sigma0 = np.eye(2)
    Sigma0[0,0] = mean_Sigma0/var_Sigma0
    Sigma0[1,1] = 1/var_Sigma0

    def m(x):
        return np.array([(1+x**2)**(-1/2)])

    def grad_m(x):
        return np.array([[-x/((1+x**2)**(3/2))]])

    model  = DSMGaussian(data=data_contaminated, m=m, grad_m=grad_m, omega=0.1, mu0=mu0, Sigma0=Sigma0)
    hazard = ConstantHazard(300)
    R_DSM = bocpd(data_contaminated, hazard, model)
    cp_DSM = find_cp(R_DSM)

    model  = Gaussian(mu0=mean0, kappa0=1, alpha0=1, omega0=1)
    hazard = ConstantHazard(300)
    R_ST = bocpd(data_contaminated, hazard, model)
    cp_ST = find_cp(R_ST)
    
    cps_DSM.append(cp_DSM)
    cps_ST.append(cp_ST)

In [150]:
accuracies = []
delays = []
for cp in cps_DSM:
    aux = accuracy_delay(cp,cps_true)
    accuracies.append(aux[0])
    delays.append(aux[1])

delays_mean= [np.mean(delay) for delay in delays]

In [151]:
print('PPV: ' + str(np.round(np.mean(np.array(accuracies)[:,0]),3))+'±'+ str(np.round(np.sqrt(np.var(np.array(accuracies)[:,0])),3)))
print('TPR: ' + str(np.round(np.mean(np.array(accuracies)[:,1]),3))+'±'+ str(np.round(np.sqrt(np.var(np.array(accuracies)[:,1])),3)))
print('Delays: '+ str(np.round(np.mean(np.array(delays_mean)),3))+'±'+ str(np.round(np.sqrt(np.var(np.array(delays_mean))),3)))

PPV: 0.907±0.154
TPR: 0.883±0.13
Delays: 1.643±0.475


In [152]:
accuracies = []
delays = []
for cp in cps_ST:
    aux = accuracy_delay(cp,cps_true)
    accuracies.append(aux[0])
    delays.append(aux[1])
delays_mean= [np.mean(delay) for delay in delays]

In [153]:
print('PPV: ' + str(np.round(np.mean(np.array(accuracies)[:,0]),3))+'±'+ str(np.round(np.sqrt(np.var(np.array(accuracies)[:,0])),3)))
print('TPR: ' + str(np.round(np.mean(np.array(accuracies)[:,1]),3))+'±'+ str(np.round(np.sqrt(np.var(np.array(accuracies)[:,1])),3)))
print('Delays: '+ str(np.round(np.mean(np.array(delays_mean)),3))+'±'+ str(np.round(np.sqrt(np.var(np.array(delays_mean))),3)))

PPV: 0.6±0.128
TPR: 0.833±0.149
Delays: 1.057±1.545
